In [ ]:
import os
import hashlib
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

def hash_file(file_path):
    """Generate a hash for a given file."""
    hash_algo = hashlib.sha256()
    try:
        with open(file_path, 'rb') as file:
            while chunk := file.read(8192):
                hash_algo.update(chunk)
        return file_path, hash_algo.hexdigest()
    except FileNotFoundError:
        print(f"\nFile not found: {file_path}")
        return file_path, None
    except PermissionError:
        print(f"\nPermission denied: {file_path}")
        return file_path, None
    except Exception as e:
        print(f"\nError processing file {file_path}: {e}")
        return file_path, None

def handle_file_conflict(src, dst):
    """Handle file conflicts by renaming the destination file."""
    base, extension = os.path.splitext(dst)
    counter = 1
    new_dst = dst
    while os.path.exists(new_dst):
        new_dst = f"{base}_{counter}{extension}"
        counter += 1
    return new_dst

def find_duplicates(directory):
    """Find and move duplicate files in a directory."""
    file_hashes = {}
    duplicates = []
    duplicate_folder = os.path.join(directory, 'duplicated_files')
    
    if not os.path.exists(duplicate_folder):
        os.makedirs(duplicate_folder)
    
    files_to_process = []
    for root, _, files in os.walk(directory):
        if duplicate_folder in root:  # Skip the duplicate folder itself
            continue
        for filename in files:
            file_path = os.path.join(root, filename)
            files_to_process.append(file_path)
    
    total_files = len(files_to_process)
    processed_files = 0

    # Adjust the number of threads as needed
    max_threads = 16  # Example value; adjust based on your system
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(hash_file, file_path) for file_path in files_to_process]
        for future in as_completed(futures):
            file_path, file_hash = future.result()
            if file_hash:
                if file_hash in file_hashes:
                    dst_path = os.path.join(duplicate_folder, os.path.basename(file_path))
                    dst_path = handle_file_conflict(file_path, dst_path)  # Handle file name conflicts
                    try:
                        shutil.move(file_path, dst_path)
                        duplicates.append(file_path)
                    except PermissionError:
                        print(f"\nPermission denied while moving file: {file_path}")
                else:
                    file_hashes[file_hash] = file_path
            
            processed_files += 1
            print(f"Processed {processed_files}/{total_files} files. Duplicates found: {len(duplicates)}", end='\r')
    
    return duplicates

def main():
    directory = input("Enter the directory to scan for duplicates: ")
    duplicates = find_duplicates(directory)
    
    if duplicates:
        print(f"\nFound and moved {len(duplicates)} duplicate files to the 'duplicated_files' folder.")
    else:
        print("\nNo duplicate files found.")

if __name__ == "__main__":
    main()
